# Constraints Demo

This notebook demonstrates the use of the *Path API* in conjunction with the *Constraint Data API*. First, we walk through the code and some simple examples. Then, we put it all together to show how you could use this technology to produce routes that respect the operational constraints of an autonomous or human fleet.

We support specifying the following constraint types, grouped into two categories:

Avoid constraints:
- **Avoid Area**: Vehicles will avoid all road segments that intersect this polygon.
- **Avoid Path**: Vehicles will avoid all road segments that are covered by this path.
- **Avoid Turn**: Vehicles will avoid the turn indicated by this path.

Operational constraints:
- **Operational Area**: Vehicles will be allowed to use roads that intersect this polygon.
- **Operational Path**: Vehicles will be allowed to use roads that are covered by this path.

Using these we can implement the following concepts for operating a fleet:
- **Operational Domain (OD)**: Expressed through operational contraints (e.g. *Operational Path* and *Operational Area*)
    - *e.g. Company XYZ is still developing their AV hardware and software, so they limit the operational domain to a network of roads throughout that city that are strategic and feasible. These roads can be identified via Operational Path constraints.*
- **Operational Design Domain (ODD)**: Expressed through avoid constraints (e.g. *Avoid Turn*)
    - *e.g. Company XYZ must limit operations even further inside their operational domain based on the capabilities which their vehicles are currently able to handle. For example, they currently must avoid all unprotected left turns (UPLs) for safety reasons. These turns can be identified via Avoid Turn constraints.*
- **Keep Out Zones (KOZs)**: Expressed through avoid constraints (e.g. *Avoid Area* and *Avoid Path*)
    - *e.g. Company XYZ must dynamically adapt to events like parades, street festivals, and construction which restricts travel along roads within their operational domain. These areas can be identified via Avoid Area constraints and be dynamically updated as frequently as necessary.*

If you wish to get to the examples and visualizations without looking through the details of the code, skip to the section called [Routing With No Constraints](#Routing-With-No-Constraints).

# Setup Code

The first snippet of code here is simply doing setup to enable the rest of the code below. One important thing to note is that running this notebook will require that you set two environment variables:
- **RIDEOS_API_KEY**: Set to the API key for your developer account at https://app.rideos.ai/profile
- **MAPBOX_ACCESS_TOKEN**: Set to an access token for Mapbox as described at https://docs.mapbox.com/help/how-mapbox-works/access-tokens/

In [ ]:
!pip install requests
!pip install mapboxgl

import json
import math
import os
import requests
import time
import uuid

from mapboxgl.viz import LinestringViz

# Load the rideOS API key
rideos_api_key = os.getenv("RIDEOS_API_KEY")
rideos_auth_header = {"X-Api-Key": rideos_api_key}

# Load the MapBox access token
mapbox_access_token = os.getenv("MAPBOX_ACCESS_TOKEN")

# Track global lists of constraints for visualization and cleanup
constraints = {
    "avoidArea": [],
    "avoidPath": [],
    "avoidTurn": [],
    "operationalArea": [],
    "operationalPath": []
}

# Add Constraint

The following method uses the *AddOrReplaceConstraints* method of the *Constraint Data API* to add a new constraint of the given type with the given geometry. A unique ID for the constraint is generated and tracked for later cleanup.

*(Note: If cleanup is not done at the end of a run of this notebook, there will be orphaned constraints left in the rideOS database for your account.)*

In [ ]:
# Create a constraint of the given type with the given points
# The type must be "avoid<Area|Path|Turn>" or "operational<Area|Path>"
# The positions should be given as a list of lat/lng tuples
# Prints the type and ID of the constraint that was added
def add_constraint(constraint_type, positions):
    if not constraint_type in constraints:
        raise Exception("" + constraint_type + "' is not a valid constraint type")
    
    # Create a string uuid that uniquely identifies this constraint
    constraint_id = str(uuid.uuid4())
    
    # Create an array of lat/lng dictionaries for the request
    request_positions = []
    for position in positions:
        request_position = {"latitude": position[0], "longitude": position[1]}
        request_positions.append(request_position)

    # Construct a constraint request
    constraint_api_url = "https://api.rideos.ai/constraints/v1/AddOrReplaceConstraints"
    constraint_request = {
        "constraints": [
            {
                "id": constraint_id,
                constraint_type: {
                    "positions": request_positions
                },
            }
        ],
    }
    
    # Print the equivalent curl command for trying out the same API call
    print("Adding constraint via API request, equivalent curl command below: ")
    print("------")
    print("curl -d '{}' -H 'Content-Type: application/json' -H 'X-Api-Key: {}' -X POST {}".format(
         json.dumps(constraint_request), rideos_api_key, constraint_api_url))
    print("-----")
    
    # Call the constraint API
    constraint_response = requests.post(
        constraint_api_url,
        headers=rideos_auth_header,
        json=constraint_request
    )
    
    # If there was an error, just print the details and continue,
    # so we can still visualize the constraint that was supposed to be added
    try:
        constraint_response.raise_for_status()
    except requests.exceptions.HTTPError as err:
        print(err)
        print(constraint_response.text)
    
    # Add the constraint ID to the corresponding global tracking list
    constraints[constraint_type].append({
        "id": constraint_id,
        "positions": positions
    })
    
    # Print the constraint type and ID
    print("Added {} constraint with ID {}".format(constraint_type, constraint_id))

# Assign Vehicle/Fleet Constraints

The following method assigns all the created constraints to the vehicle identified in the constant above. This function isn't used in our examples below, but this method can be used to assign constraints to a particular vehicle. 

A similar function (using `/ride-hail-operations/v1/SetFleetRoutingConstraints`) can be used to assign constraints to an entire fleet.

In [ ]:
# Assign constraints to the given vehicle ID
# The vehicle ID must be an existing vehicle within the account for the current API key
def assign_vehicle_constraints(vehicle_id):
    # Identify the constraint IDs for operational and avoid constraints
    operational_constraint_ids = []
    avoid_constraint_ids = []
    for constraint_type, constraint_list in constraints.items():
        if constraint_type.startswith("operational"):
            for constraint in constraint_list:
                operational_constraint_ids.append(constraint["id"])
        else:
            for constraint in constraint_list:
                avoid_constraint_ids.append(constraint["id"])    
    
    # Construct a vehicle constraint request
    vehicle_api_url = "https://api.rideos.ai/ride-hail-operations/v1/SetVehicleRoutingConstraints"
    vehicle_request = {
        "vehicleId": vehicle_id,
        "constraints": {
            # these variables will be renamed to operationalConstraintIds and avoidConstraintIds soon
            "whitelistConstraintIds": operational_constraint_ids,
            "blacklistConstraintIds": avoid_constraint_ids
        },
    }
    
    # Print the equivalent curl command for trying out the same API call
    print("Assigning constraints to vehicle via API request, equivalent curl command below: ")
    print("------")
    print("curl -d '{}' -H 'Content-Type: application/json' -H 'X-Api-Key: {}' -X POST {}".format(
         json.dumps(vehicle_request), rideos_api_key, vehicle_api_url))
    print("-----")
    
    # Call the vehicle constraint API
    vehicle_response = requests.post(
        vehicle_api_url,
        headers=rideos_auth_header,
        json=vehicle_request
    )
    vehicle_response.raise_for_status()

    # Print the constraint type and ID
    print("Assigned {} constraint(s) to vehicle {}".format(
        len(operational_constraint_ids) + len(avoid_constraint_ids), vehicle_id))

# Clear Constraints

The following method uses the *DeleteConstraints* method of the *Constraint Data API* to delete all the constraints which have been created in the notebook so far. This uses the IDs tracked above to provide reliable cleanup.

In [ ]:
# Delete all constraints which were created within this run of the notebook
# Prints the number of constraints which were deleted
def clear_constraints():
    # Get all of the constraint IDs
    constraint_ids = []
    for constraint_type, constraint_list in constraints.items():
        for constraint in constraint_list:
            constraint_ids.append(constraint["id"])
    
    # Construct a delete request
    delete_api_url = "https://api.rideos.ai/constraints/v1/DeleteConstraints"
    delete_request = {
        "ids": constraint_ids
    }
    
    # Print the equivalent curl command for trying out the same API call
    print("Clearing constraints via API request, equivalent curl command below: ")
    print("------")
    print("curl -d '{}' -H 'Content-Type: application/json' -H 'X-Api-Key: {}' -X POST {}".format(
         json.dumps(delete_request), rideos_api_key, delete_api_url))
    print("-----")
    
    # Call the delete API
    delete_response = requests.post(
        delete_api_url,
        headers=rideos_auth_header,
        json=delete_request
    )
    delete_response.raise_for_status()
    
    # Clear the constraint lists
    for constraint_type, constraint_list in constraints.items():
        del constraint_list[:]
    
    # Print the number of deleted constraints
    print("Deleted {} constraint(s)".format(len(constraint_ids)))

# Get Path and Visualize

The methods below use the *GetPath* method of the *Path API* to fetch a path, then visualize it on a map. The path is computed between the given start and end points, and it will respect any constraints that were created (using methods below) and not yet cleared.

If any **operational constraints** are present, then anything outside of those operational constraints will be avoided. If any **avoid constraints** are present, then anything inside one of those avoid constraints will be avoided. The avoid constraints override the operational constraints.

The visualization will show the path and all of the constraints colored by type.

In [ ]:
# Get the GeoJSON representing a given list of positions
# The positions should be lat/lng tuples
# Return a GeoJSON linestring object as a dictionary
def get_linestring_geojson(positions):
    geojson = {
        "type": "LineString",
        "coordinates": []
    }
    for position in positions:
        geojson_coords = (position[1], position[0])
        geojson["coordinates"].append(geojson_coords)
    return geojson


# Get the GeoJSON approximately representing a circle around the given point
# The center should be a lat/lng tuple
# The radius is in meters (but the result is approximate)
# The number of points determines how many points will be sampled along the circumference
# Return a GeoJSON linestring object as a dictionary
def get_circle_geojson(center, radius = 3, num_points = 10):
    geojson = {
        "type": "LineString",
        "coordinates": []
    }
    for i in xrange(num_points):
        angle = math.pi * 2 * i / num_points
        dx = radius * math.cos(angle)
        dy = radius * math.sin(angle)
        lat = center[0] + (180 / math.pi) * (dy / 6378137)
        lng = center[1] + (180 / math.pi) * (dx / 6378137) / math.cos(center[0] * math.pi / 180)
        geojson_coords = (lng, lat)
        geojson["coordinates"].append(geojson_coords)
    return geojson


# Get the center of the minimal bounding box containing the given positions
# The positions should be lat/lng tuples
# Return a lat/lng tuple representing the center of the bounding box
def get_center_of_bounding_box(positions):
    # If there are no positions, just return a trivial result
    if len(positions) == 0:
        return (0, 0)
    
    # Track the edges of the minimal bounding box
    min_lat = 90
    max_lat = -90
    min_lng = 180
    max_lng = -180
    
    # Find the minimal bounding box 
    for position in positions:
        min_lat = min(min_lat, position[0])
        max_lat = max(max_lat, position[0])
        min_lng = min(min_lng, position[1])
        max_lng = max(max_lng, position[1])
    
    # Return the center of the minimal bounding box
    return ((min_lat + max_lat) / 2, (min_lng + max_lng) / 2)


# Fetch and visualize the path from start to end, respecting any current constraints
# Start and end should be lat/lng tuples
def visualize_path(start, end, zoomLevel = 13):
    # Identify the constraint IDs for operational and avoid constraints
    operational_constraint_ids = []
    avoid_constraint_ids = []
    for constraint_type, constraint_list in constraints.items():
        if constraint_type.startswith("operational"):
            for constraint in constraint_list:
                operational_constraint_ids.append(constraint["id"])
        else:
            for constraint in constraint_list:
                avoid_constraint_ids.append(constraint["id"])
    
    # Construct a path request
    path_api_url = "https://api.rideos.ai/path/v2/GetPath"
    path_request = {
        "waypoints": [
            {"position": {"latitude": start[0], "longitude": start[1]}},
            {"position": {"latitude": end[0], "longitude": end[1]}}
        ],
        "geometryFormat": "LINESTRING",
        "operationalConstraintIds": operational_constraint_ids,
        "avoidConstraintIds": avoid_constraint_ids
    }
    
    # Print the equivalent curl command for trying out the same API call
    print("Getting path via API request, equivalent curl command below: ")
    print("------")
    print("curl -d '{}' -H 'Content-Type: application/json' -H 'X-Api-Key: {}' -X POST {}".format(
         json.dumps(path_request), rideos_api_key, path_api_url))
    print("-----")

    # Fetch the path response and parse the JSON
    path_response = requests.post(
        path_api_url,
        headers=rideos_auth_header,
        json=path_request
    )
    path_response.raise_for_status()
    path_response_json = path_response.json()
    print("Successfully found path, visualizing below")
    
    # Extract the positions from the path response
    positions = []
    for path in path_response_json["paths"]:
        for leg in path["legs"]:
            for position in leg["lineString"]["positions"]:
                positions.append((position["latitude"], position["longitude"]))

    # Get the center of the bounding box and convert to GeoJSON coords (lng/lat)
    center = get_center_of_bounding_box(positions)
    center_geojson_coords = (center[1], center[0])

    # Construct the GeoJSON for the path and constraints
    geojson = {
       "type": "FeatureCollection",
       "features": []
    }
    for constraint_type, constraint_list in constraints.items():
        for constraint in constraint_list:
            geojson["features"].append({
                "type": "Feature",
                "id": constraint["id"],
                "properties": {"type": constraint_type},
                "geometry": get_linestring_geojson(constraint["positions"])
            })
    geojson["features"].append({
        "type": "Feature",
        "id": "path",
        "properties": {"type": "path"},
        "geometry": get_linestring_geojson(positions)
    })
    geojson["features"].append({
        "type": "Feature",
        "id": "origin",
        "properties": {"type": "origin"},
        "geometry": get_circle_geojson(positions[0])
    })
    geojson["features"].append({
        "type": "Feature",
        "id": "destination",
        "properties": {"type": "destination"},
        "geometry": get_circle_geojson(positions[-1])
    })

    # Visualize the path in MapBox, centered at the center of the bounding box
    viz = LinestringViz(geojson,
                        access_token = mapbox_access_token,
                        center = center_geojson_coords,
                        color_function_type = "match",
                        color_property = "type",
                        color_stops = [
                            ["path", "#0000ff"],
                            ["origin", "#9cf196"],
                            ["destination", "#ff4893"],
                            ["avoidArea", "#ff0000"],
                            ["avoidPath", "#ff6200"],
                            ["avoidTurn", "#ffbf00"],
                            ["operationalArea", "#009900"],
                            ["operationalPath", "#00cc00"]
                        ],
                        line_width_function_type = "match",
                        line_width_property = "type",
                        line_width_stops = [
                            ["path", 3],
                            ["origin", 10],
                            ["destination", 10],
                            ["avoidArea", 5],
                            ["avoidPath", 5],
                            ["avoidTurn", 5],
                            ["operationalArea", 5],
                            ["operationalPath", 5]
                        ],
                        opacity = 0.8,
                        zoom = zoomLevel)
    viz.show()

In [ ]:
visualize_path((37.766043, -122.405925), (37.764778, -122.427317), zoomLevel = 13)

# Setting an Avoid Area

Now we add an *avoidArea* constraint, which will force the route to travel around the area defined by the given polygon.

In [ ]:
clear_constraints()

add_constraint("avoidArea", [
    (37.765337, -122.411971), (37.765406, -122.410998), (37.765796, -122.411048),
    (37.765746, -122.412034), (37.765337, -122.411971)])

Now we attempt to find the re-routed path.

It takes some time for new constraints to propagate through the rideOS services, but that time is typically less than 15 seconds. We use a sleep statement here and below to allow for this propagation before attempting to re-route. If the path does not avoid the *avoidArea* constrained roads, try re-running just this visualize step again. Perhaps the constraint needed more time to propagate.

In [ ]:
print("Waiting 2 minutes for constraints to propagate...")
time.sleep(120)

visualize_path((37.766043, -122.405925), (37.764778, -122.427317), zoomLevel = 13)

# Setting an Avoid Path

Now we clear the constraints, and add an *avoidPath* constraint, which will force the route to travel around the road that matches the given path most closely.

In [ ]:
clear_constraints()

add_constraint("avoidPath", [
    (37.765766, -122.408105), (37.765387, -122.414523)])

Now we attempt to find the re-routed path.

If the path does not avoid the *avoidPath* constrained roads, try re-running just this visualize step again. Perhaps the constraint needed more time to propagate.

In [ ]:
print("Waiting 2 minutes for constraints to propagate...")
time.sleep(120)

visualize_path((37.766043, -122.405925), (37.764778, -122.427317), zoomLevel = 13)

# Setting an Avoid Turn

First, let us consider a route that includes a right turn.

In [ ]:
clear_constraints()

visualize_path((37.766043, -122.405925), (37.766425, -122.410431), zoomLevel = 13)

Now we consider an *avoidTurn* constraint, which will force the route to travel around the turn through an intersection that matches the given angle most closely.

In [ ]:
add_constraint("avoidTurn", [
    (37.765627, -122.410249), (37.765636, -122.410506), (37.765814, -122.410538)])

Now we attempt to find the re-routed path.

If the path does not avoid the *avoidTurn* constrained turn, try re-running just this visualize step again. Perhaps the constraint needed more time to propagate.

In [ ]:
print("Waiting 2 minutes for constraints to propagate...")
time.sleep(120)

visualize_path((37.766043, -122.405925), (37.766425, -122.410431), zoomLevel = 13)

# Setting an Operational Area

First, let us consider a route that should be a straight path.

In [ ]:
clear_constraints()

visualize_path((37.76465, -122.41540), (37.759418, -122.414847), zoomLevel = 14)

Now we add an *operationalArea* constraint, which will force the route to travel within the area defined by the given polygon.

In [ ]:
add_constraint("operationalArea", [
    (37.76597,-122.41455), (37.76563,-122.41592), (37.76431,-122.41592),
    (37.76336,-122.42438), (37.75780,-122.42390), (37.75827,-122.41395),
    (37.76126,-122.41439), (37.76071,-122.42198), (37.76258,-122.42233),
    (37.76319,-122.41430),
    (37.76597,-122.41455)
])

Now we attempt to find the re-routed path.

If the path does not stay within the *operationalArea* roads, try re-running just this visualize step again. Perhaps the constraint needed more time to propagate.

In [ ]:
print("Waiting 2 minutes for constraints to propagate...")
time.sleep(120)

visualize_path((37.76465, -122.41540), (37.759418, -122.414847), zoomLevel = 14)

# Setting an Operational Path

Now we clear the constraints, and add an *operationalPath* constraint, which will allow the route to travel along the road that matches the given path most closely.

In [ ]:
clear_constraints()

add_constraint("operationalPath", [
    (37.76518,-122.41533), (37.76370,-122.41517), (37.76376,-122.41365),
    (37.76215,-122.41408), (37.76221,-122.41285), (37.76060,-122.41270),
    (37.76050,-122.41490), (37.75811,-122.41466)
])

Now we attempt to find the re-routed path.

If the path does not use the *operationalPath* road as a shortcut, try re-running just this visualize step again. Perhaps the constraint needed more time to propagate.

In [ ]:
print("Waiting 2 minutes for constraints to propagate...")
time.sleep(120)

visualize_path((37.76465, -122.41540), (37.759418, -122.414847), zoomLevel = 14)

# Implementing a Complex Operational Domain

In the real world, an operational domain is composed of a number of directed roads and possibly some dense areas as well. Combining operational and avoid constraints together can produce an arbitrarily complex operational domain. Below, we define an example using a set of *operationalPath* constraints for each directed road.

In [ ]:
clear_constraints()

add_constraint("operationalArea", [
    (37.76597,-122.41455), (37.76563,-122.41592), (37.76431,-122.41592),
    (37.76336,-122.42438), (37.75780,-122.42390), (37.75827,-122.41395),
    (37.76126,-122.41439), (37.76071,-122.42198), (37.76258,-122.42233),
    (37.76319,-122.41430),
    (37.76597,-122.41455)
])

add_constraint("avoidPath", [
    (37.76140,-122.42389), (37.76027,-122.42375)])

#assign_vehicle_constraints("some_vehicle_id")

Now we attempt to find some paths within the operational domain.

If the paths do not stay within the operational domain, try re-running just this visualize step again. Perhaps the constraints needed more time to propagate.

In [ ]:
print("Waiting 2 minutes for constraints to propagate...")
time.sleep(120)

visualize_path((37.76465, -122.41540), (37.759418, -122.414847), zoomLevel = 14)

visualize_path((37.759418, -122.414847), (37.76465, -122.41540), zoomLevel = 14)

# Clean Up

Now that we're all done, let's clean up all the constraints we created, so we don't leave orphaned test constraints in the rideOS database.

In [ ]:
clear_constraints()
#assign_vehicle_constraints("some_vehicle_id")